In [7]:
from exportaciones import EXPORTACIONES
import pandas as pd

In [8]:
xs = EXPORTACIONES()

# xs.donwload_data()

In [9]:
exp = xs.filter()
subsectores = list(exp.subsector.unique())
entidades = list(exp.entidad.unique())

In [10]:
exp_g = (exp.groupby(["year", "entidad","subsector"]).agg({"monto":"sum"}).loc["2022"]).reset_index()

In [22]:
exp.entidad = exp.entidad.apply(lambda x: x.capitalize())
exp.entidad.unique()

array(['Aguascalientes', 'Baja california', 'Baja california sur',
       'Campeche', 'Coahuila de zaragoza', 'Colima', 'Chiapas',
       'Chihuahua', 'Ciudad de méxico', 'Durango', 'Guanajuato',
       'Guerrero', 'Hidalgo', 'Jalisco', 'México', 'Michoacán de ocampo',
       'Morelos', 'Nayarit', 'Nuevo león', 'Oaxaca', 'Puebla',
       'Querétaro', 'Quintana roo', 'San luis potosí', 'Sinaloa',
       'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala',
       'Veracruz de ignacio de la llave', 'Yucatán', 'Zacatecas'],
      dtype=object)

In [17]:
def crecimiento_nacional(df):
    # df = df[(df[])]
    df = df.groupby("year").agg({"monto":"sum"}).pct_change()*100
    return df

crecimiento_nacional(exp)

,monto
year,
2007,NaN
2008,8.476528
2009,-23.155472
2010,30.403773
2011,15.948558
2012,6.766574
2013,2.983787
2014,5.460867
2015,-2.989268


In [8]:
def gen_group_nacional(df):
    df = df[df["year"] == "2022"]
    df = df.groupby(["subsector"]).agg({"monto":"sum"})

    return df

gen_group_nacional(exp)

,monto
subsector,
Agricultura,15496.072
Cría y explotación de animales,1004.455
"Curtido y acabado de cuero y piel, y fabricación de productos de cuero, piel y materiales sucedáneos",1310.176
Extracción de petróleo y gas,31865.120
"Fabricación de accesorios, aparatos eléctricos y equipo de generación de energía eléctrica",31226.293
"Fabricación de equipo de computación, comunicación, medición y de otros equipos, componentes y accesorios electrónicos",92253.000
Fabricación de equipo de transporte,185242.807
Fabricación de insumos textiles y acabado de textiles,936.513
Fabricación de maquinaria y equipo,23664.756


In [10]:
def ranking_subsector(df, subsector):
    df = df[(df["year"] == "2022") & (df["subsector"] == subsector)]

    df = df.groupby(["entidad"]).agg({"monto":"sum"})
    df["subsector"] = subsector
    df["ranking_nacional"] = df["monto"].rank(ascending=False)

    
    return df

df_ranking = pd.DataFrame()

for s in subsectores:
    temp = ranking_subsector(exp, s)
    df_ranking = pd.concat([df_ranking, temp])

df_ranking = df_ranking.reset_index()
df_ranking

,entidad,monto,subsector,ranking_nacional
0,Aguascalientes,108.662,Subsectores no especificados,11.0
1,Baja California,53.124,Subsectores no especificados,20.0
2,Baja California Sur,5.945,Subsectores no especificados,30.0
3,Campeche,9.463,Subsectores no especificados,28.0
4,Chiapas,234.144,Subsectores no especificados,9.0
...,...,...,...,...
859,Tamaulipas,2271.266,Otras industrias manufactureras,3.0
860,Tlaxcala,0.000,Otras industrias manufactureras,23.5
861,Veracruz de Ignacio de la Llave,0.000,Otras industrias manufactureras,23.5
862,Yucatán,351.034,Otras industrias manufactureras,7.0


In [23]:
def group_entidad(df, df_rank, entidad):
    df = df[(df["year"] == "2022") & (df["entidad"] == entidad)]
    df_rank = df_rank[df_rank["entidad"] == entidad]

    df = df.groupby("subsector").agg({"monto":"sum"})
    total = df.sum()
    
    df["%"] = (df/total)*100

    df.columns = ["Exportaciones", "% en el total estatal"]
    df = df.sort_values("Exportaciones", ascending = False)

    df = df.merge(df_rank, left_index=True, right_on="subsector")

    df = df[["subsector","Exportaciones", "% en el total estatal", "ranking_nacional"]]

    return df

group_entidad(exp,df_ranking, "Hidalgo")



,subsector,Exportaciones,% en el total estatal,ranking_nacional
780,Fabricación de equipo de transporte,1588.887,48.100166,14.0
12,Subsectores no especificados,1055.983,31.967633,3.0
620,Industrias metálicas básicas,144.194,4.365166,12.0
428,Industria del papel,132.032,3.996987,9.0
268,Fabricación de insumos textiles y acabado de t...,111.264,3.368280,4.0
204,Industria alimentaria,75.980,2.300132,23.0
556,Industria del plástico y del hule,69.753,2.111623,18.0
332,Fabricación de prendas de vestir,34.648,1.048894,15.0
524,Industria química,30.745,0.930739,20.0
748,"Fabricación de accesorios, aparatos eléctricos...",22.818,0.690766,14.0


In [26]:
writer = pd.ExcelWriter("exportaciones.xlsx")
for entidad in entidades:
    temp = group_entidad(exp,df_ranking, entidad)
    temp.to_excel(writer, sheet_name=entidad, index=False)

writer.close()